##### Import libraries

In [40]:
import pdb
import os
import shutil

In [1]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf

In [9]:
import sklearn.preprocessing

###### Preprocess training samples of a section

In [58]:
def fn_normalizeJobsSectionSamples(strPart, strJobsDir, strJobsSamplesDir):
    strJobsSamplesFile = os.path.join(strJobsSamplesDir, "sections/" + strPart + "/samples.csv")
    pdDfJobsSamples = pd.read_csv(strJobsSamplesFile)
    oMinMaxScaler = sklearn.preprocessing.MinMaxScaler()
    oMinMaxScaler.fit(pdDfJobsSamples.values)
    for name in os.listdir(strJobsDir):
        strJobDir = os.path.join(strJobsDir, name)
        strSectionSamplesFile = os.path.join(strJobDir, "samples/sections/" + strPart + "/samples.csv")
        pdDfSectionSamples = pd.read_csv(strSectionSamplesFile)
        npNArrPreprocessedSectionSamples = oMinMaxScaler.transform(pdDfSectionSamples.values)
        pdDfPreprocessedSectionSamples = pd.DataFrame(npNArrPreprocessedSectionSamples, columns=pdDfSectionSamples.columns)
        strPreprocessedSectionSamplesDir = os.path.join(strJobDir, "samples/sections/" + strPart + "/preprocessed")
        if os.path.exists(strPreprocessedSectionSamplesDir):
            shutil.rmtree(strPreprocessedSectionSamplesDir)
        os.mkdir(strPreprocessedSectionSamplesDir)
        strPreprocessedSectionSamplesFile = os.path.join(strPreprocessedSectionSamplesDir, "samples.csv")
        pdDfPreprocessedSectionSamples.to_csv(strPreprocessedSectionSamplesFile, index = False)
    return oMinMaxScaler

##### Generate training dataset
Preprocess data before training instead on the fly. So there is no preprocess callable object in the list of input parameters.

In [49]:
def fn_getTrainingDataset(strPart, strJobsDir, 
                          nReaders = 5, nReadThreads = 5, nParseTreads = 5, nShuffleBufferSize = 1000, nBatchSize = 32):
    strSampleFiles = fn_getTrainingSampleFiles(strPart, strJobsDir)
    oDataset = tf.data.Dataset.list_files("jobs/*/samples/sections/" + strPart + "/preprocessed/samples.csv")
    oDataset = oDataset.interleave(lambda strSampleFile: tf.data.TextLineDataset(strSampleFile), cycle_length=nReaders, 
                                  num_parallel_calls=nReadThreads)
    oDataset = oDataset.shuffle(nShuffleBufferSize)
    return oDataset.batch(nBatchSize)

## Input

In [60]:
oMinMaxScalerInput = fn_normalizeJobsSectionSamples("input", "jobs", "samples")

In [26]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow import math

import numpy as np

g_nBatchSize = 5

def fn_DLoss(tensorTrue, tensorPred):
        tensorOnes, tensorReal = tensorTrue[:g_nBatchSize, :], tensorPred[:g_nBatchSize, :]
        tensorZeros, tensorFake = tensorTrue[g_nBatchSize:, :], tensorPred[g_nBatchSize:, :]
        tensorLossReal = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorReal, labels = tensorOnes), 1)
        tensorLossFake = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorFake, labels = tensorZeros), 1)
        tensorLoss = tensorLossReal + tensorLossFake
        return math.reduce_mean(tensorLoss)  
def fn_GLoss(tensorTrue, tensorPred):
    tensorLoss = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorPred, labels = tensorTrue), 1)
    return math.reduce_mean(tensorLoss)

class GANOperator(object):
    """
        specify the number of features to be dealt by gan
    """
    def __init__(self, nLatentDim, nHiddenDimG, nHiddenDimD, nBatchSize, nFeatures):
        self.nLatentDim = nLatentDim
        self.nHiddenDimG = nHiddenDimG
        self.nHiddenDimD = nHiddenDimD
        self.nBatchSize = nBatchSize
        self.nFeatures = nFeatures
        
        self.oSeqGenerator = None
        self.oSeqDiscriminator = None
        self.fn_makeGenerator()
        self.fn_makeDiscriminator()
        self.fn_makeDiscriminatorModel()
        self.fn_makeAdversariaModel()
        
    def fn_makeGenerator(self):
        if self.oSeqGenerator:
            return self.oSeqGenerator
        self.oSeqGenerator = Sequential()
        self.oSeqGenerator.add(Dense(self.nHiddenDimG, activation = tf.keras.activations.tanh))
        self.oSeqGenerator.add(Dense(self.nFeatures, activation = tf.keras.activations.tanh))
    def fn_makeDiscriminator(self):
        if self.oSeqDiscriminator:
            return self.oSeqDiscriminator
        self.oSeqDiscriminator = Sequential()
        self.oSeqDiscriminator.add(Dense(self.nHiddenDimD, activation = tf.keras.activations.tanh))
        self.oSeqDiscriminator.add(Dense(1, activation = tf.keras.activations.sigmoid))
    
    def fn_makeDiscriminatorModel(self):
        self.oSeqDiscriminatorModel = Sequential()
        self.oSeqDiscriminatorModel.add(self.oSeqDiscriminator)
        oOptimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1)
        self.oSeqDiscriminatorModel.compile(loss=fn_DLoss, optimizer=oOptimizer, metrics=["accuracy"])
    def fn_makeAdversariaModel(self):
        self.oSeqAdversarialModel = Sequential()
        self.oSeqAdversarialModel.add(self.oSeqGenerator)
        self.oSeqAdversarialModel.add(self.oSeqDiscriminator)
        self.oSeqDiscriminator.trainable = False
        oOptimer = tf.optimizers.Adam()
        self.oSeqAdversarialModel.compile(loss = fn_GLoss, optimizer=oOptimer, metrics=["accuracy"])
    
    def fn_train(self, npNArrSamples, nEpochs = 2000):
        for epoch in range(nEpochs):
            npNArrTrueSamples = npNArrSamples[np.random.randint(0, npNArrSamples.shape[0], size = self.nBatchSize), :]
            npNoise = np.random.uniform(-1.0, 1.0, size = [self.nBatchSize, self.nLatentDim])
            npNArrFake = self.oSeqGenerator.predict(npNoise)
            npNArrX = np.concatenate((npNArrTrueSamples, npNArrFake))
            npNArrY = np.ones([2 * self.nBatchSize, 1])
            npNArrY[self.nBatchSize:, :] = 0
            fLossD = self.oSeqDiscriminatorModel.train_on_batch(npNArrX, npNArrY)
            
            npNArrY = np.ones([self.nBatchSize, 1])
            npNoise = np.random.uniform(-1.0, 1.0, size = [self.nBatchSize, self.nLatentDim])
            fLossA = self.oSeqAdversarialModel.train_on_batch(npNoise, npNArrY)

            strMsg = "%d: [D loss: %f, acc: %f]" % (epoch, fLossD[0], fLossD[1])
            strMsg = "%s [A loss: %f, acc: %f]" % (strMsg, fLossA[0], fLossA[1])
            print(strMsg)

In [31]:
keras.backend.clear_session()
oGANOperator = GANOperator(nLatentDim=8, nHiddenDimG=16, nHiddenDimD=16, nBatchSize=g_nBatchSize, 
                           nFeatures=5)

In [32]:
oGANOperator.fn_train(npNArrSamples, nEpochs=10)

0: [D loss: 1.452051, acc: 0.400000] [A loss: 0.490367, acc: 0.200000]
1: [D loss: 1.434356, acc: 0.500000] [A loss: 0.474354, acc: 0.400000]
2: [D loss: 1.459817, acc: 0.400000] [A loss: 0.508515, acc: 0.000000]
3: [D loss: 1.454618, acc: 0.500000] [A loss: 0.449057, acc: 0.800000]
4: [D loss: 1.487272, acc: 0.300000] [A loss: 0.479009, acc: 0.400000]
5: [D loss: 1.391144, acc: 0.800000] [A loss: 0.503868, acc: 0.200000]
6: [D loss: 1.450166, acc: 0.500000] [A loss: 0.478879, acc: 0.400000]
7: [D loss: 1.487112, acc: 0.400000] [A loss: 0.463928, acc: 0.800000]
8: [D loss: 1.436899, acc: 0.500000] [A loss: 0.446789, acc: 0.600000]
9: [D loss: 1.470820, acc: 0.300000] [A loss: 0.479095, acc: 0.400000]
